# Modeling and Predictions

[Notebook 1: EDA and Cleaning](./1_EDA and Cleaning.ipynb)

[Notebook 2: Modeling and Predictions](./2_Modeling and Predictions.ipynb)

1. Features
2. Resampling
3. Modeling
    - Scaling
    - Lagged features
    - Train test split, fit models, evaluate

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta

import time

import matplotlib
import seaborn as sns
%config InlineBackend.figure_format = 'retina'

from tqdm import tqdm

from sklearn.metrics import mean_squared_error, r2_score
from numpy import sqrt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler

In [ ]:
matplotlib.rcParams['figure.figsize'] = (13, 8)

In [ ]:
# Read in pickle.
data = pd.read_pickle('./pickle.pkl')

# Features

In [ ]:
# Add hours feature
data['Hour'] = data.index.hour

In [ ]:
# Get features
features = ['DOY', 'DNI', 'Air Temp', 'Hour', 'Humidity', 'Pressure', 'Wind Speed', 'Wind Dir']
label = ['DNI']

# Resampling
Resampling data to reduce runtime. Resampling during the period where the battery did not hold a charge, we see the  missing raw data.

In [ ]:
data_re = data[features].resample('60T').mean()

In [ ]:
data_re.sort_index(inplace=True)

In [ ]:
print(data_re[data_re['DNI'].isnull()])

In [ ]:
data_re.dropna(axis=0, inplace=True)

# Modeling

## Scaling

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(data_re)
scaled = pd.DataFrame(scaled, index=data_re.index, columns=data_re.columns)

## Lagged features

In [ ]:
# Create df of all lagged features

lags = [24, 72, 168]
lstm_data = pd.DataFrame()
lagged_feat_dict = {}

for x in lags:
    
    # shift
    lagged_data = scaled[features].shift(periods=x).dropna() # drop after looping
    
    # rename lagged features
    lagged_col_names = {}
    for y in lagged_data.columns:
        lagged_col_names[y] = y + '_' + str(x)
    lagged_data.rename(columns=lagged_col_names, inplace=True)
    
    # make dict of lagged_col_names (to reference for later)
    lagged_feat_dict[x] = lagged_col_names.values()
        
    # make lstm_data the lagged data
    lstm_data = pd.concat([lstm_data, lagged_data], axis=1)
    
# Add DNI target
lstm_data = pd.concat([lstm_data, scaled[label]], axis=1)

## Train test split, fit models, evaluate

In [ ]:
year = 2012

tts = len(lstm_data.loc[lstm_data.index.year < year])

model_results = []

epochs_ = 2
batch_size_ = 1000
dropout_ = .3

for x in lags:
    
    print('Training w/ lag', x, 'hours:')
    
    feats = list(lagged_feat_dict[x])
    
    lstm_temp = lstm_data[feats + ['DNI']].dropna()
    lstm_train = lstm_temp.loc[lstm_temp.index.year < year]
    lstm_test = lstm_temp.loc[lstm_temp.index.year >= year]
    
    X_train = lstm_train[feats].values
    X_test = lstm_test[feats].values
    y_train = lstm_train['DNI'].values
    y_test = lstm_test['DNI'].values
    
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
        
    # design network
    model = Sequential()
    model.add(LSTM(int(x), input_shape=(X_train.shape[1], X_train.shape[2]))) # num cells
    model.add(Dropout(dropout_))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # fit network
    history = model.fit(X_train, 
                        y_train, 
                        epochs=epochs_, 
                        batch_size=batch_size_, 
                        validation_data=(X_test, y_test), 
                        verbose=1, 
                        shuffle=False)
    
    # Save model for later
    filename = str(int(time.time())) + '_model_' + str(x) + '_lag.h5'
    model.save('./models/' + filename)
    
    # plot history
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.title('Training ' + str(x) + ' hours lag')
    plt.show()

    # make a prediction
    yhat = model.predict(X_test)
    
    # Concat y_hat to dataframe for later
    lstm_test = pd.concat([lstm_test, pd.DataFrame(yhat, columns=['yhat'], index=lstm_test.index)], axis=1)
    
    # Invert scaling (so we can get our forecast and evaluation in terms of DNI (W/m2))

    # Get yhat and y_test
    yhat = lstm_test['yhat']
    original = lstm_test['DNI']

    # Copy lstm_test
    lstm_test_yhat = lstm_test.iloc[:, :8]
    lstm_test_original = lstm_test.iloc[:, :8]

    # Substitute yhat and y_test into lstm_test
    lstm_test_yhat.iloc[:, 1] = yhat
    lstm_test_original.iloc[:, 1] = original

    # Inverse yhat
    inv_yhat = scaler.inverse_transform(lstm_test_yhat)
    inv_yhat = pd.DataFrame(inv_yhat).iloc[:, 1]

    # Inverse y_test
    inv_y_test = scaler.inverse_transform(lstm_test_original)
    inv_y_test = pd.DataFrame(inv_y_test).iloc[:, 1]
    
    # Average RMSE, in terms of DNI (W/m2)
    print('RMSE in DNI (W/m2): %.3f' % sqrt(mean_squared_error(inv_y_test, inv_yhat)))
          
    hours = 300

    plt.plot(yhat[-hours:], label='yhat')
    plt.plot(original[-hours:], label='y test')
    plt.legend()
    plt.title('Y test vs. y hat, ' + str(x) + ' hours lag')
    
    plt.show()
    
    # # calculate RMSE
    rmse = sqrt(mean_squared_error(yhat, y_test))
    r2 = r2_score(yhat, y_test)
    print(str(x) + ' hours lag test R2 score: %.3f' % r2)
    print(str(x) + ' hours lag test RMSE: %.3f' % rmse)
    print()
    
    # Save all the results
    
    model_results_dict = {}
    
    model_results_dict['tts_year'] = year
    model_results_dict['lag'] = x
    model_results_dict['dropout'] = dropout_
    model_results_dict['epochs'] = epochs_
    model_results_dict['batch_size'] = batch_size_
    model_results_dict['params'] = history.params
    model_results_dict['loss'] = history.history
    model_results_dict['rmse'] = rmse
    model_results_dict['dni_rmse'] = sqrt(mean_squared_error(inv_y_test, inv_yhat))
    model_results_dict['r2'] = r2
    model_results_dict['model_filename'] = filename
    model_results_dict['time_ran'] = int(time.time())
    
    model_results.append(model_results_dict)
    
# Reads in old results and concats with new results

new_res_df = pd.DataFrame(model_results)
old_res_df = pd.read_csv('./results/results.csv', index_col=0)
res_df = pd.concat([old_res_df, new_res_df], axis=0, sort=False).reset_index(drop=True)
res_df.to_csv('./results/results.csv')

In [ ]:
res_df